In [57]:
from sqlalchemy import create_engine, func
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
import numpy as np
import datetime as dt
from pprint import pprint

In [2]:
# Create engine
engine = create_engine("sqlite:///database/hawaii.sqlite")

# Declare Base
Base = automap_base()

# Use Base class to reflect database tables
Base.prepare(engine, reflect=True)

# Save references to measurements and stations tables
Measurement = Base.classes.measurements
Station = Base.classes.stations

# Create a session
session = Session(engine)

In [74]:
session.query(Measurement.date, Measurement.prcp).\
    filter(Measurement.date >= "2017-01-01").all()

# Convert list of tuples into normal list
# prcp_list = list(np.ravel(results))
    
# print(results)

[('2017-01-01', 0),
 ('2017-01-02', 0),
 ('2017-01-03', 0),
 ('2017-01-04', 0),
 ('2017-01-05', 0),
 ('2017-01-06', 0),
 ('2017-01-07', 0),
 ('2017-01-08', 0),
 ('2017-01-09', 0),
 ('2017-01-10', 0),
 ('2017-01-11', 0),
 ('2017-01-12', 0),
 ('2017-01-13', 0),
 ('2017-01-14', 0),
 ('2017-01-15', 0),
 ('2017-01-16', 0),
 ('2017-01-17', 0),
 ('2017-01-18', 0),
 ('2017-01-19', 0),
 ('2017-01-20', 0),
 ('2017-01-21', 0),
 ('2017-01-22', 0.16),
 ('2017-01-23', 0),
 ('2017-01-24', 0.04),
 ('2017-01-25', 0.03),
 ('2017-01-26', 0),
 ('2017-01-27', 0),
 ('2017-01-28', 0),
 ('2017-01-29', 0.18),
 ('2017-01-30', 0),
 ('2017-01-31', 0),
 ('2017-02-01', 0),
 ('2017-02-02', 0),
 ('2017-02-03', 0),
 ('2017-02-04', 0),
 ('2017-02-05', 0),
 ('2017-02-06', 0),
 ('2017-02-07', 0.51),
 ('2017-02-08', 0),
 ('2017-02-09', 0),
 ('2017-02-10', 0),
 ('2017-02-11', 0.31),
 ('2017-02-12', 2.62),
 ('2017-02-13', 0.01),
 ('2017-02-14', 0),
 ('2017-02-15', 0),
 ('2017-02-16', 0.07),
 ('2017-02-17', 0),
 ('2017-02-18

In [5]:
result = session.query(Station).first()
result.__dict__

{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState at 0x181a80bb6a0>,
 'elevation': 3.0,
 'id': 1,
 'latitude': 21.2716,
 'longitude': -157.8168,
 'name': 'WAIKIKI 717.2, HI US',
 'station': 'USC00519397'}

In [7]:
session.query(Station.station, Station.name, Station.latitude, Station.longitude, Station.elevation).all()

[('USC00519397', 'WAIKIKI 717.2, HI US', 21.2716, -157.8168, 3.0),
 ('USC00513117', 'KANEOHE 838.1, HI US', 21.4234, -157.8015, 14.6),
 ('USC00514830',
  'KUALOA RANCH HEADQUARTERS 886.9, HI US',
  21.5213,
  -157.8374,
  7.0),
 ('USC00517948', 'PEARL CITY, HI US', 21.3934, -157.9751, 11.9),
 ('USC00518838', 'UPPER WAHIAWA 874.3, HI US', 21.4992, -158.0111, 306.6),
 ('USC00519523',
  'WAIMANALO EXPERIMENTAL FARM, HI US',
  21.33556,
  -157.71139,
  19.5),
 ('USC00519281', 'WAIHEE 837.5, HI US', 21.45167, -157.84888999999995, 32.9),
 ('USC00511918', 'HONOLULU OBSERVATORY 702.2, HI US', 21.3152, -157.9992, 0.9),
 ('USC00516128', 'MANOA LYON ARBO 785.2, HI US', 21.3331, -157.8025, 152.4)]

In [75]:
def daily_normals(a_date):
    """
    Grab min, avg, max temperatures for historic dates.
    """

    # Query to grab all historic temperatures by date
    temps = session.query(Measurement.tobs).\
        filter(Measurement.date.like(f"%{a_date}")).all()

    # Convert list of tuples into normal list
    temps = list(np.ravel(temps))

    # Calculate min, avg, and max temps
    min_temp = min(temps)
    avg_temp = np.mean(temps).round(1).item()
    max_temp = max(temps)

    return min_temp, avg_temp, max_temp

In [71]:
daily_normals("8-01")

(67, 75.599999999999994, 83)

In [72]:
def calc_temps(start):
    # Query precipitation data from last year
    results = session.query(Measurement.date).\
        filter(Measurement.date >= start).all()
    
    # Convert list of tuples into normal list
    date_list = list(np.ravel(results))
    
    # Create month day list
    month_day_list = []

    for date in date_list:

        # Convert datetime to string
        a_date = dt.datetime.strptime(date, "%Y-%m-%d")

        # Grab only month and day
        month_day = f"{dt.datetime.strftime(a_date, '%m-%d')}"

        # Append month_day to list
        month_day_list.append(month_day)
    
    # Create lists to hold min, avg, and max temperatures
    min_temp_list = []
    avg_temp_list = []
    max_temp_list = []

    # Loop through each date in trip date list
    for date in month_day_list:

        # Pass date into daily_normals function
        min_temp, avg_temp, max_temp = daily_normals(date)

        # Append returns to lists
        min_temp_list.append(min_temp)
        avg_temp_list.append(avg_temp)
        max_temp_list.append(max_temp)

    # Create a dictionary from the row data and append to a list of temp_data
    temp_data = []
    for i, date in enumerate(date_list):
        row = {}
        row["date"] = date
        row["tmin"] = min_temp_list[i]
        row["tavg"] = avg_temp_list[i]
        row["tmax"] = max_temp_list[i]
        temp_data.append(row)
    
    pprint(temp_data) 

In [76]:
calc_temps("2017-03-01")

[{'date': '2017-03-01', 'tavg': 70.0, 'tmax': 76, 'tmin': 64},
 {'date': '2017-03-02', 'tavg': 70.2, 'tmax': 77, 'tmin': 61},
 {'date': '2017-03-03', 'tavg': 70.2, 'tmax': 78, 'tmin': 60},
 {'date': '2017-03-04', 'tavg': 69.2, 'tmax': 77, 'tmin': 57},
 {'date': '2017-03-05', 'tavg': 68.1, 'tmax': 76, 'tmin': 60},
 {'date': '2017-03-06', 'tavg': 69.1, 'tmax': 76, 'tmin': 60},
 {'date': '2017-03-07', 'tavg': 69.1, 'tmax': 78, 'tmin': 60},
 {'date': '2017-03-08', 'tavg': 68.5, 'tmax': 78, 'tmin': 59},
 {'date': '2017-03-09', 'tavg': 68.5, 'tmax': 75, 'tmin': 57},
 {'date': '2017-03-10', 'tavg': 68.9, 'tmax': 75, 'tmin': 57},
 {'date': '2017-03-11', 'tavg': 69.8, 'tmax': 82, 'tmin': 60},
 {'date': '2017-03-12', 'tavg': 69.7, 'tmax': 77, 'tmin': 62},
 {'date': '2017-03-13', 'tavg': 69.3, 'tmax': 77, 'tmin': 56},
 {'date': '2017-03-14', 'tavg': 69.9, 'tmax': 77, 'tmin': 62},
 {'date': '2017-03-15', 'tavg': 68.3, 'tmax': 76, 'tmin': 58},
 {'date': '2017-03-16', 'tavg': 69.4, 'tmax': 78, 'tmin